In [1]:
import os
import cv2
import numpy as np
from sklearn.metrics import roc_curve, auc
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, ConcatDataset, random_split
from tqdm import tqdm
from sklearn.svm import SVC

In [2]:
class PalmROIDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.img_list = sorted([f for f in os.listdir(root_dir) if f.endswith(".bmp")])
        self.transform = transform

    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):
        img_name = self.img_list[idx]
        img_path = os.path.join(self.root_dir, img_name)
        image = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if image is None:
            raise ValueError(f"Failed to load image: {img_path}")
        image = cv2.resize(image, (32, 32))
        image = image[..., np.newaxis]  # Add channel dimension (32, 32, 1)
        image = image.astype(np.float32) / 255.0  # Convert to float32 and normalize
        image = torch.from_numpy(image).permute(2, 0, 1)  # Convert to tensor (1, 32, 32)
        if self.transform:
            image = self.transform(image)
        # Extract person ID (e.g., '00001.bmp' to '00020.bmp' -> person 1)
        try:
            img_number = int(img_name.split(".")[0])  # Get numeric part (1 to 12000)
            person_id = ((img_number - 1) // 20) + 1  # Map to person 1 to 300
            label = person_id - 1  # 0-based indexing (0 to 299)
            if not (0 <= label <= 299):
                raise ValueError(f"Label {label + 1} out of range (1 to 300) for {img_name}")
        except (ValueError, IndexError):
            raise ValueError(f"Invalid filename format for label: {img_name}")
        return image, label, img_name

In [3]:
# LeNet feature extractor
class LeNetFeature(nn.Module):
    def __init__(self):
        super(LeNetFeature, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.AvgPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        features = F.relu(self.fc2(x))
        return features

In [4]:
# Function to extract features from a dataloader
def extract_features(model, dataloader, device):
    model.eval()
    features_list = []
    labels_list = []
    with torch.no_grad():
        for x, y, _ in tqdm(dataloader, desc="Extracting features"):
            x = x.to(device)
            f = model(x).cpu().numpy()  
            features_list.append(f)
            labels_list.extend(y.numpy())
    features = np.concatenate(features_list, axis=0)
    labels = np.array(labels_list)
    return features, labels

In [5]:
# Compute metrics (Top-1, Top-5, ROC AUC, EER, FAR, FRR)
def compute_metrics(features, labels, clf):
    # Get decision scores for all test samples
    decision_scores = clf.decision_function(features)  

    # Top-1 and Top-5 accuracy
    topk_acc = {}
    for k in [1, 5]:
        top_k_indices = np.argsort(decision_scores, axis=1)[:, -k:]  
        correct = 0
        for i, top_k in enumerate(top_k_indices):
            if labels[i] in top_k:
                correct += 1
        topk_acc[k] = correct / len(labels)

    # ROC-related metrics (pairwise comparison using decision score differences)
    y_true, y_score = [], []
    for i in range(len(labels)):
        for j in range(i + 1, len(labels)):
            # Use maximum decision score difference as similarity
            score_i = decision_scores[i, labels[i]]
            score_j = decision_scores[j, labels[j]]
            sim = score_i + score_j 
            y_score.append(sim)
            y_true.append(1 if labels[i] == labels[j] else 0)

    # Check if y_true has both classes
    if len(set(y_true)) <= 1:
        print("Warning: y_true contains only one class. Setting ROC AUC, EER, FAR, FRR to 0.")
        roc_auc = 0.1
        eer = 0.1
        far = 0.1
        frr = 0.1
    else:
        fpr, tpr, thresholds = roc_curve(y_true, y_score)
        roc_auc = auc(fpr, tpr)
        fnr = 1 - tpr
        diff = np.abs(fnr - fpr)
        if np.all(np.isnan(diff)):
            print("Warning: All-NaN slice in EER calculation. Setting EER, FAR, FRR to 0.1")
            eer = 0.1
            far = 0.1
            frr = 0.1
        else:
            eer_idx = np.nanargmin(diff)
            eer = (fpr[eer_idx] + fnr[eer_idx]) / 2
            far = fpr[eer_idx]
            frr = fnr[eer_idx]

    return topk_acc, roc_auc, eer, far, frr

In [6]:
# Main code
session1_root = r"C:\Users\hiteshk\Desktop\Deep Learning Approaches for roi extraction and using same for palm print recognisation\Tonji\ROI\session1"
session2_root = r"C:\Users\hiteshk\Desktop\Deep Learning Approaches for roi extraction and using same for palm print recognisation\Tonji\ROI\session2"

In [7]:
# Create datasets
dataset1 = PalmROIDataset(session1_root)
dataset2 = PalmROIDataset(session2_root)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
print(f"Session 1 images: {len(dataset1)}")
print(f"Session 2 images: {len(dataset2)}")

Session 1 images: 6000
Session 2 images: 6000


In [10]:
full_dataset = ConcatDataset([dataset1, dataset2])
print(f"Total images: {len(full_dataset)}")

Total images: 12000


In [11]:
# Split into train and test (80/20)
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [12]:
#Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [13]:
# Initialize model
model = LeNetFeature().to(device)

In [14]:
# Extract features
train_features, train_labels = extract_features(model, train_loader, device)
test_features, test_labels = extract_features(model, test_loader, device)

Extracting features: 100%|██████████| 38/38 [00:00<00:00, 42.33it/s]


In [15]:
# Verify number of classes
num_classes = len(np.unique(np.concatenate((train_labels, test_labels))))
print(f"Number of classes: {num_classes}")
if num_classes != 300:
    print(f"Warning: Expected 300 classes, but found {num_classes}. Check filename labels.")

Number of classes: 300


In [16]:
# Train SVM classifier
svm_clf = SVC(kernel='rbf', decision_function_shape='ovr', probability=False)
svm_clf.fit(train_features, train_labels)

SVC()

In [17]:
topk_acc, roc_auc, eer, far, frr = compute_metrics(test_features, test_labels, svm_clf)

In [18]:
print(f"Top-1 Identification Accuracy: {topk_acc[1]*100:.2f}%")
print(f"Top-5 Identification Accuracy: {topk_acc[5]*100:.2f}%")
print(f"ROC AUC: {roc_auc:.4f}")
print(f"EER: {eer:.4f}")
print(f"FAR: {far:.4f}")
print(f"FRR: {frr:.4f}")

Top-1 Identification Accuracy: 0.38%
Top-5 Identification Accuracy: 2.62%
ROC AUC: 0.4138
EER: 0.5674
FAR: 0.5674
FRR: 0.5673
